### Simulando a coleta de dados com Spark Streaming através da "escuta" em uma porta TCP-IP especificada
#### Para isso, iremos usar o netcat como ferramenta de apoio.
porta escolhida: 22121

comando: nc -lk 22121

## 1. Importando os módulos necessários para o Streaming de Dados

In [1]:
#fonte: https://github.com/danielsan/Spark-Streaming-Examples/blob/master/spark-streaming-foreachRDD-and-foreach.py
# Módulos do Spark

from pyspark.streaming import StreamingContext
from pyspark import SparkContext # quando usamos o PYSPARK, o SPARK CONTEXT já é criado por default: sc

## 2. Criando o contexto com o Spark Streaming
- Lembrando que o contexto com a aplicação Spark, por default pelo PYSPARK já é criado automaticamente com o nome "sc"

In [2]:
print('-->> Verificando o contexto em que se encontra a conexão:', sc) #sc = spark context
print('-->> Versão do SPARK em execução:', sc.version)

# Definindo o contexto do Streaming de dados com Spark, uma vez que o contexto com o Spark já foi criado por default
strcontext = StreamingContext(sparkContext = sc, batchDuration = 1)

-->> Verificando o contexto em que se encontra a conexão: <SparkContext master=local[*] appName=PySparkShell>
-->> Versão do SPARK em execução: 2.2.0


## 3. Criando o RECEIVER do Spark.
- No caso, estamos usando o socketTextStream por se tratar de uma conexão à uma porta TCP-IP
- A coleta de dados é possível através do Twitter, Apache Flume, Apache Kafka, HDFS do Hadoop, IOT: ou seja, as fontes de dados para o RECEIVER que irá "alimentar" o Streaming do Spark. Veja, são inúmeras.

In [3]:
# Criando o RECEIVER para fazer o streaming de dados TCP-IP = socketTextStream
hostname = "localhost"
port = 22121

lines = strcontext.socketTextStream(hostname = hostname, port = port)
print("Type object 'lines':", lines)

Type object 'lines': <pyspark.streaming.dstream.DStream object at 0x7f60000c1c50>


## 4. Tratamento e Tranformação
### 4.1. Para cada linha, divide as palavras a cada " " (espaço) encontrado

In [4]:
# como estamos executando função de transforamção sobre o DSTREAM gerado (lines), então devemos "jogar" o resultado
# da transformação em um novo DSTREAM, pois este é sempre IMUTÁVEL.
words = lines.flatMap(lambda lines : lines.split(" "))

print("Type object 'words':", words)

Type object 'words': <pyspark.streaming.dstream.TransformedDStream object at 0x7f60000c1748>


### 4.2. Conta o número de ocorrências das palavras em cada batch entregue pelo streaming de dados

In [5]:
pairs = words.map(lambda words : (words, 1))
# Exemplo de saída: (('ciência', 1), ('Big Data', 2), ('abacaxi', 1))

wordCounts = pairs.reduceByKey(lambda x, y: x + y) # onde a chave é a própria palavra!
print("Type object 'wordCounts':", wordCounts)

Type object 'wordCounts': <pyspark.streaming.dstream.TransformedDStream object at 0x7f60000c5240>


## 5. Imprimindo os 10 primeiros elementos de cada RDD gerado no DStream
RDD = Resilient Distributed Dataset

In [6]:
wordCounts.pprint()

## 6. Funções para as etapas de:
- Criação do Receiver para o Streaming de dados.
- Tranformação dos dados ainda no DStream
- Persistência dos dados do DStream para RDD + ações de redução

In [ ]:
import pandas as pd

# Módulos do Spark
from pyspark.streaming import StreamingContext
#from pyspark import SparkContext # quando usamos o PYSPARK, o SPARK CONTEXT já é criado por default: sc

print('-->> Verificando o contexto em que se encontra a conexão:', sc) #sc = spark context
print('-->> Versão do SPARK em execução:', sc.version)

# Definindo o contexto do Streaming de dados com Spark, uma vez que o contexto com o Spark já foi criado por default
strcontext = StreamingContext(sparkContext = sc, batchDuration = 1)

hostname = "localhost"
port = 22121

# Criando uma lista vazia
values = list()

def ReceiverDataStreaming(hostname, port):
    '''
    Especificação da função...
    '''
    # Criando o RECEIVER no Streaming Context
    lines = strcontext.socketTextStream(hostname = hostname, port = port)
    return(lines)

def TransformationDataStreaming():
    '''
    Especificação da função...
    '''
    # Chamando a função para construção do RECEIVER
    lines = ReceiverDataStreaming(hostname = hostname, port = port)

    # Para cada linha, divide as palavras a cada " " (espaço) encontrado
    words = lines.flatMap(lambda lines : lines.split(" "))

    # Conta o número de ocorrências das palavras em cada batch entregue pelo streaming de dados
    pairs = words.map(lambda words : (words, 1))
    wordsCount = pairs.reduceByKey(lambda x, y: x + y) # onde a chave é a própria palavra!
    return(wordsCount)

def sendRecord(tup):
    '''
    Especificação da função...
    '''
    word   = tup[0]
    amount = tup[1]
    content = (word, amount)
    
    #values.append(content)

def PersistDSTream(DStream):
    '''
    Objetivo: persistir os dados do DSTREAM em uma tupla.
    
    A parâmetro de entrada será sempre o microbatching gerado pelo DSTREAM (que nada mais é que uma micro coleção
    de dados (RDD)!), onde é gerado pela função TransformationDataStreaming().
    '''
    DStream.foreachRDD(lambda rdd : rdd.foreach(sendRecord))

'''
def WordsCountByKeyRDD(rdd):
    PersistDSTreamToRDD(rdd = TransformationDataStreaming())    
    RDD_input = rdd.foreach(lambda record : strcontext.send(record))
    #RDD_f = RDD_app.map(lambda x: x).sortBy(lambda x : x[1], ascending = False).collect()
    return(type(RDD_input))
'''

### 6.1. Executando as funções declaradas...

In [ ]:
PersistDSTream(DStream = TransformationDataStreaming())

## 7. Início e encerramento da coleta do stream de dados

- strcontext.start() = Iniciando a coleta e processamento do stream de dados.
- strcontext.awaitTermination() = a coleta de dados por streaming irá rodar indefinidamente até que encontre um erro de execução ou caso finalize todo o trabalho de streaming de dados.

In [ ]:
strcontext.start()
strcontext.awaitTermination()

-------------------------------------------
Time: 2018-04-22 22:09:20
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:09:21
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:09:22
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:09:23
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:09:24
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:09:25
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:09:26
-------------------------------------------
('aasdasd', 1)
('a', 8)
('asdas', 1)

-------------------------------------------
Time: 2018-04-22 22:09:27
-------------------------------------------

-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:10:31
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:10:32
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:10:33
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:10:34
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:10:35
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:10:36
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:10:37
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:10:38
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 22:10:39
----------

### 7.1 Encerrando o RECEIVER...

In [ ]:
strcontext.stop()